In [1]:
basic_app = '''import streamlit as st
import joblib

st.title("Customer Churn Predictor")
st.write("Testing basic app")

try:
    model = joblib.load('churn_prediction_model.pkl')
    st.success("Model loaded!")
except:
    st.error("Model not found")'''

with open('app.py', 'w') as f:
    f.write(basic_app)

print("Basic app created!")

Basic app created!


In [2]:
# Check if basic app was created
import os
if os.path.exists('app.py'):
    print("✅ Basic app.py created successfully!")
    print(f"File size: {os.path.getsize('app.py')} bytes")
else:
    print("❌ app.py not created")

✅ Basic app.py created successfully!
File size: 246 bytes


In [ ]:
import streamlit as st
import pandas as pd
import joblib
import numpy as np

# Set page configuration
st.set_page_config(
    page_title="Customer Churn Predictor",
    page_icon="📊",
    layout="wide"
)

# Load the saved model and columns
@st.cache_resource
def load_model():
    model = joblib.load('churn_prediction_model.pkl')
    model_columns = joblib.load('model_columns.pkl')
    return model, model_columns

# Main function
def main():
    st.title("🏢 Customer Churn Prediction App")
    st.markdown("""
    **Predict whether a customer will leave your service using AI/ML**
    
    *This model analyzes customer behavior and service usage patterns to predict churn risk with 79% accuracy*
    """)
    
    # Load model
    try:
        model, model_columns = load_model()
        st.success("✅ Model loaded successfully!")
    except Exception as e:
        st.error(f"❌ Error loading model: {e}")
        st.info("Please make sure 'churn_prediction_model.pkl' and 'model_columns.pkl' are in the same directory")
        return
    
    # Create input form
    st.header("📝 Customer Information")
    
    col1, col2, col3 = st.columns(3)
    
    with col1:
        st.subheader("💰 Demographic & Billing")
        tenure = st.slider("Tenure (months)", 0, 72, 12, 
                          help="How long the customer has been with the company")
        monthly_charges = st.number_input("Monthly Charges ($)", 0.0, 200.0, 70.0, 0.1,
                                         help="Customer's monthly payment")
        total_charges = st.number_input("Total Charges ($)", 0.0, 10000.0, 1000.0, 0.1,
                                       help="Total amount paid by customer")
        
    with col2:
        st.subheader("👤 Personal Information")
        gender = st.selectbox("Gender", ["Female", "Male"])
        senior_citizen = st.selectbox("Senior Citizen", ["No", "Yes"])
        partner = st.selectbox("Partner", ["No", "Yes"])
        dependents = st.selectbox("Dependents", ["No", "Yes"])
        
    with col3:
        st.subheader("📞 Service Information")
        internet_service = st.selectbox("Internet Service", ["DSL", "Fiber optic", "No"])
        contract = st.selectbox("Contract Type", ["Month-to-month", "One year", "Two year"])
        payment_method = st.selectbox("Payment Method", [
            "Bank transfer (automatic)", "Credit card (automatic)", 
            "Electronic check", "Mailed check"
        ])
        paperless_billing = st.selectbox("Paperless Billing", ["No", "Yes"])
    
    # Additional services
    st.subheader("🔧 Additional Services")
    col4, col5 = st.columns(2)
    
    with col4:
        st.write("**Communication Services**")
        phone_service = st.selectbox("Phone Service", ["No", "Yes"])
        multiple_lines = st.selectbox("Multiple Lines", ["No", "No phone service", "Yes"])
        
        st.write("**Security Services**")
        online_security = st.selectbox("Online Security", ["No", "No internet service", "Yes"])
        online_backup = st.selectbox("Online Backup", ["No", "No internet service", "Yes"])
        
    with col5:
        st.write("**Protection Services**")
        device_protection = st.selectbox("Device Protection", ["No", "No internet service", "Yes"])
        tech_support = st.selectbox("Tech Support", ["No", "No internet service", "Yes"])
        
        st.write("**Streaming Services**")
        streaming_tv = st.selectbox("Streaming TV", ["No", "No internet service", "Yes"])
        streaming_movies = st.selectbox("Streaming Movies", ["No", "No internet service", "Yes"])
    
    # Divider
    st.markdown("---")
    
    # Prediction button
    if st.button("🔮 Predict Churn Probability", type="primary", use_container_width=True):
        
        with st.spinner("Analyzing customer data..."):
            # Create input dataframe
            input_data = {
                'tenure': tenure,
                'MonthlyCharges': monthly_charges,
                'TotalCharges': total_charges,
                'is_new_customer': 1 if tenure <= 12 else 0,
                'is_long_term_customer': 1 if tenure > 24 else 0,
                'is_high_spender': 1 if monthly_charges > 70 else 0,
                'gender_Male': 1 if gender == "Male" else 0,
                'SeniorCitizen_Yes': 1 if senior_citizen == "Yes" else 0,
                'Partner_Yes': 1 if partner == "Yes" else 0,
                'Dependents_Yes': 1 if dependents == "Yes" else 0,
                'PhoneService_Yes': 1 if phone_service == "Yes" else 0,
                'MultipleLines_No phone service': 1 if multiple_lines == "No phone service" else 0,
                'MultipleLines_Yes': 1 if multiple_lines == "Yes" else 0,
                'InternetService_Fiber optic': 1 if internet_service == "Fiber optic" else 0,
                'InternetService_No': 1 if internet_service == "No" else 0,
                'OnlineSecurity_No internet service': 1 if online_security == "No internet service" else 0,
                'OnlineSecurity_Yes': 1 if online_security == "Yes" else 0,
                'OnlineBackup_No internet service': 1 if online_backup == "No internet service" else 0,
                'OnlineBackup_Yes': 1 if online_backup == "Yes" else 0,
                'DeviceProtection_No internet service': 1 if device_protection == "No internet service" else 0,
                'DeviceProtection_Yes': 1 if device_protection == "Yes" else 0,
                'TechSupport_No internet service': 1 if tech_support == "No internet service" else 0,
                'TechSupport_Yes': 1 if tech_support == "Yes" else 0,
                'StreamingTV_No internet service': 1 if streaming_tv == "No internet service" else 0,
                'StreamingTV_Yes': 1 if streaming_tv == "Yes" else 0,
                'StreamingMovies_No internet service': 1 if streaming_movies == "No internet service" else 0,
                'StreamingMovies_Yes': 1 if streaming_movies == "Yes" else 0,
                'Contract_One year': 1 if contract == "One year" else 0,
                'Contract_Two year': 1 if contract == "Two year" else 0,
                'PaperlessBilling_Yes': 1 if paperless_billing == "Yes" else 0,
                'PaymentMethod_Credit card (automatic)': 1 if payment_method == "Credit card (automatic)" else 0,
                'PaymentMethod_Electronic check': 1 if payment_method == "Electronic check" else 0,
                'PaymentMethod_Mailed check': 1 if payment_method == "Mailed check" else 0
            }
            
            # Create DataFrame with all model columns (fill missing with 0)
            input_df = pd.DataFrame([input_data])
            
            # Ensure all model columns are present
            for col in model_columns:
                if col not in input_df.columns:
                    input_df[col] = 0
            
            # Reorder columns to match training
            input_df = input_df[model_columns]
            
            # Make prediction
            prediction = model.predict(input_df)[0]
            prediction_proba = model.predict_proba(input_df)[0]
            
        # Display results
        st.header("🎯 Prediction Results")
        
        result_col1, result_col2, result_col3 = st.columns([2,1,1])
        
        with result_col1:
            if prediction == 1:
                st.error("## 🚨 HIGH RISK: Customer Likely to CHURN")
                st.write("**Immediate action required to retain this customer**")
            else:
                st.success("## ✅ LOW RISK: Customer Likely to STAY")
                st.write("**Continue providing excellent service**")
        
        with result_col2:
            st.metric(
                "Churn Probability", 
                f"{prediction_proba[1]*100:.1f}%",
                delta=f"{(prediction_proba[1]-0.5)*100:+.1f}%",
                delta_color="inverse"
            )
        
        with result_col3:
            risk_level = "High" if prediction_proba[1] > 0.7 else "Medium" if prediction_proba[1] > 0.4 else "Low"
            st.metric("Risk Level", risk_level)
        
        # Probability gauge
        st.subheader("📈 Risk Assessment")
        st.progress(float(prediction_proba[1]))
        
        # Key factors analysis
        st.subheader("🔍 Key Risk Factors")
        
        # Analyze key factors based on our earlier insights
        risk_factors = []
        
        if tenure <= 12:
            risk_factors.append("🆕 **New customer** (≤ 12 months tenure)")
        if monthly_charges > 70:
            risk_factors.append("💸 **High spender** (> $70 monthly)")
        if contract == "Month-to-month":
            risk_factors.append("📄 **Month-to-month contract**")
        if payment_method == "Electronic check":
            risk_factors.append("🧾 **Electronic check payment**")
        if online_security == "No" and internet_service != "No":
            risk_factors.append("🛡️ **No online security**")
        
        if risk_factors:
            for factor in risk_factors:
                st.write(f"- {factor}")
        else:
            st.write("- No major risk factors identified")
        
        # Interpretation and recommendations
        st.subheader("💡 Insights & Recommendations")
        
        if prediction_proba[1] > 0.7:
            st.warning("### 🚨 HIGH RISK CUSTOMER")
            st.write("**Immediate retention actions recommended:**")
            st.write("🎯 **Offer loyalty discount** or promotional rate")
            st.write("📞 **Personal outreach** from customer success team")
            st.write("🔍 **Review service quality** and address any issues")
            st.write("📊 **Conduct satisfaction survey** to identify pain points")
            st.write("💝 **Exclusive benefits** or premium feature trial")
            
        elif prediction_proba[1] > 0.4:
            st.info("### ⚠️ MEDIUM RISK CUSTOMER")
            st.write("**Proactive monitoring and engagement:**")
            st.write("📅 **Regular check-ins** every 2-3 months")
            st.write("⭐ **Ensure service satisfaction** and address concerns")
            st.write("🎁 **Consider proactive offers** or value-added services")
            st.write("📋 **Gather feedback** for service improvement")
            st.write("🔔 **Set up alerts** for usage pattern changes")
            
        else:
            st.success("### ✅ LOW RISK CUSTOMER")
            st.write("**Maintain and strengthen relationship:**")
            st.write("🌟 **Continue excellent service** delivery")
            st.write("📈 **Consider upselling opportunities**")
            st.write("🤝 **Build long-term partnership**")
            st.write("💡 **Gather feedback** for continuous improvement")
            st.write("🎉 **Reward loyalty** with exclusive benefits")
    
    # Sidebar with project info
    with st.sidebar:
        st.title("About This Project")
        st.markdown("""
        **AI/ML Customer Churn Prediction**
        
        This application uses machine learning to predict customer churn risk based on:
        
        - **Demographic data**
        - **Service usage patterns** 
        - **Billing information**
        - **Contract details**
        
        **Model Performance:**
        - Accuracy: 79%
        - Precision: 82%
        - Recall: 89%
        
        **Key Insights:**
        - New customers (≤12 months) have higher churn risk
        - High spenders (>$70/month) are more likely to leave
        - Long-term contracts reduce churn risk
        
        *Built with Streamlit & Scikit-learn*
        """)
        
        st.markdown("---")
        st.markdown("**Developer:** Your Name  ")
        st.mark("**Portfolio Project**")

# Run the app
if __name__ == "__main__":
    main()